# Any and all pacakges used for cleaning and modeling

In [1]:
import pandas as pd
# !pip install wrds
import wrds
import os
import zipfile

c:\Users\ryant\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Part 1: Merging SPAC and IPO data
- Key Points: Self cleaned excel files with offering dates and firm names
- Both read in seperately and merged on the date variabe

In [2]:
spac_data = pd.read_csv("inputs/SPACs2016-2021.csv", header=0)
IPO_data = pd.read_csv("inputs/IPO_data.csv", header=0)


In [3]:
IPO_data['CUSIP'] = IPO_data['CUSIP'].apply(lambda x: str(x).zfill(9))
IPO_data

,Unnamed: 0,offer date,IPO name,CUSIP
0,0,01/30/1975,ROYSTER,078088610
1,1,06/09/1975,VARCO,092212610
2,2,06/10/1975,COORS ADOLPH,021701610
3,3,07/15/1975,KEYSTONE FOODS CORP,049348410
4,4,08/26/1975,C. F. BRAUN,010564710
...,...,...,...,...
15442,15442,12/14/2023,INNO HOLDINGS INC.,4576JP109
15443,15443,12/15/2023,Bayview Acquisition,000000nan
15444,15444,12/15/2023,ZKH GROUP LTD,98877R104
15445,15445,12/19/2023,Linkage Global Inc,G5500B102


In [4]:
IPO_data = IPO_data[IPO_data['CUSIP'] != '000000nan']
IPO_data

,Unnamed: 0,offer date,IPO name,CUSIP
0,0,01/30/1975,ROYSTER,078088610
1,1,06/09/1975,VARCO,092212610
2,2,06/10/1975,COORS ADOLPH,021701610
3,3,07/15/1975,KEYSTONE FOODS CORP,049348410
4,4,08/26/1975,C. F. BRAUN,010564710
...,...,...,...,...
15440,15440,12/01/2023,Garden Stage Ltd,G3730L107
15441,15441,12/04/2023,Aimei Health Technology Co.,G01341117
15442,15442,12/14/2023,INNO HOLDINGS INC.,4576JP109
15444,15444,12/15/2023,ZKH GROUP LTD,98877R104


In [5]:
spac_data['CUSIP'] = spac_data['CUSIP'].apply(lambda x: str(x).zfill(9))
spac_data

,SPAC IPO,Date of IPO,CUSIP
0,Aetherium Acquisition Corp.,12/29/2021,00809J200
1,Welsbach Technology Metals Acquisition Corp.,12/27/2021,950415208
2,Arogo Capital Acquisition Corp.,12/23/2021,042644203
3,Gardiner Healthcare Acquisitions Corp.,12/21/2021,365506203
4,Larkspur Health Acquisition Corp.,12/20/2021,51724W206
...,...,...,...
1009,CF Corp.,5/25/2016,000000nan
1010,KLR Energy Acquisition Corp.,3/16/2016,777385204
1011,Jensyn Acquisition Corp.,3/7/2016,47632B201
1012,Silver Run Acquisition Corp.,2/29/2016,82811P200


In [6]:
spac_data = spac_data[spac_data['CUSIP'] != '000000nan']
spac_data

,SPAC IPO,Date of IPO,CUSIP
0,Aetherium Acquisition Corp.,12/29/2021,00809J200
1,Welsbach Technology Metals Acquisition Corp.,12/27/2021,950415208
2,Arogo Capital Acquisition Corp.,12/23/2021,042644203
3,Gardiner Healthcare Acquisitions Corp.,12/21/2021,365506203
4,Larkspur Health Acquisition Corp.,12/20/2021,51724W206
...,...,...,...
1007,M III Acquisition Corp.,7/7/2016,55378T203
1008,"Landcadia Holdings, Inc.",6/1/2016,51476W206
1010,KLR Energy Acquisition Corp.,3/16/2016,777385204
1011,Jensyn Acquisition Corp.,3/7/2016,47632B201


In [7]:
merged_IPOnSPAC = pd.merge(left=IPO_data, right=spac_data, how='left', on='CUSIP')
merged_IPOnSPAC['SPAC IPO '] = merged_IPOnSPAC['SPAC IPO '].notna().astype(int)
merged_IPOnSPAC = merged_IPOnSPAC.drop(merged_IPOnSPAC.columns[-1], axis=1)
merged_IPOnSPAC.to_csv('inputs/MergedIPO&SPAC.csv')



In [13]:
merged_IPOnSPAC

,Unnamed: 0,offer date,IPO name,CUSIP,SPAC IPO
0,0,01/30/1975,ROYSTER,078088610,0
1,1,06/09/1975,VARCO,092212610,0
2,2,06/10/1975,COORS ADOLPH,021701610,0
3,3,07/15/1975,KEYSTONE FOODS CORP,049348410,0
4,4,08/26/1975,C. F. BRAUN,010564710,0
...,...,...,...,...,...
15318,15440,12/01/2023,Garden Stage Ltd,G3730L107,0
15319,15441,12/04/2023,Aimei Health Technology Co.,G01341117,0
15320,15442,12/14/2023,INNO HOLDINGS INC.,4576JP109,0
15321,15444,12/15/2023,ZKH GROUP LTD,98877R104,0


In [8]:
# # Convert 'offer date' from 'YYYYMMDD' to 'MM/DD/YYYY'
# IPO_data['offer date'] = pd.to_datetime(IPO_data['offer date'],format='%Y%m%d')
# IPO_data['offer date'] = IPO_data['offer date'].dt.strftime('%m/%d/%Y')

# IPO_data.to_csv("inputs/IPO_data.csv")



In [9]:
# IPO_data.rename(columns={'offer date' : 'Date of IPO'}, inplace=True)

C:\Users\ryant\AppData\Local\Temp\ipykernel_1552\270019101.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  IPO_data.rename(columns={'offer date' : 'Date of IPO'}, inplace=True)


In [11]:
#Reading in ccm data
ccm_youngfirms = "inputs/ccm_youngfirms_2000_2018.dta/ccm_youngfirms_2000_2018.dta"

if not os.path.exists(ccm_youngfirms):
    zip_path = "inputs/ccm_youngfirms_2000_2018.zip"

    with zipfile.ZipFile(zip_path,'r') as zip_ref:
        zip_ref.extractall("inputs/ccm_youngfirms_2000_2018.dta")
        
ccm_data = data = pd.read_stata(ccm_youngfirms)

In [15]:

merged_IPOnSPAC.rename(columns={'CUSIP' : 'cusip'}, inplace=True)
merge_df_ccm = pd.merge(merged_IPOnSPAC, ccm_data, on='cusip', how='inner')

merge_df_ccm

,Unnamed: 0,offer date,IPO name,cusip,SPAC IPO,lpermno,lpermco,gvkey,datadate,fyear,...,prcc_f,prch_f,prcl_f,adjex_f,rank,au,auop,auopic,ceoso,cfoso
0,2089,06/05/1986,Genzyme,372917104,0,10324.0,8243.0,012233,2000-12-31,2000,...,89.9375,103.750,39.68750,2.0000,1.0,7,4,,,
1,2089,06/05/1986,Genzyme,372917104,0,10324.0,8243.0,012233,2001-12-31,2001,...,59.8600,64.000,34.34375,1.0000,1.0,7,1,,,
2,2089,06/05/1986,Genzyme,372917104,0,10324.0,8243.0,012233,2002-12-31,2002,...,29.5700,58.550,15.64000,1.0000,1.0,7,4,,,
3,4619,06/16/1992,Columbia Banking System,197236102,0,86004.0,16035.0,065228,2000-12-31,2000,...,8.5000,10.000,5.50000,0.5958,NaN,,,,,
4,4619,06/16/1992,Columbia Banking System,197236102,0,86004.0,16035.0,065228,2001-12-31,2001,...,13.5000,14.600,8.18750,0.5958,NaN,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10499,14676,07/13/2021,"NorthEast Community Bancorp, I",664121100,0,91405.0,58620.0,175083,2006-12-31,2006,...,12.2900,12.350,10.75000,1.3400,1.0,9,1,0,Y,Y
10500,14676,07/13/2021,"NorthEast Community Bancorp, I",664121100,0,91405.0,58620.0,175083,2007-12-31,2007,...,11.8300,12.888,9.25000,1.3400,1.0,9,1,0,Y,Y
10501,14676,07/13/2021,"NorthEast Community Bancorp, I",664121100,0,91405.0,58620.0,175083,2008-12-31,2008,...,6.9400,12.500,6.00000,1.3400,1.0,9,1,0,Y,Y
10502,14676,07/13/2021,"NorthEast Community Bancorp, I",664121100,0,91405.0,58620.0,175083,2009-12-31,2009,...,6.5700,9.490,6.10000,1.3400,1.0,9,1,0,Y,Y
